# The Pokemon Cookbook
This cookbook teaches you the concepts of the InfluxDB 3.0 Python Client library using a novel example of Pokemon data. The scenerio is to keep track of each trainer and the number of different pokemon they have caught.

<p align="center">
<img height="300" src="https://www.nicepng.com/png/full/62-622961_no-one-knows-if-people-eat-pokmon-png.png">
</p>

In [ ]:

# Here we include all the imports required from influxdb_client_3 
import pandas as pd
import random
from pyarrow.flight import FlightDescriptor, FlightClient
from pyarrow import flight
# These are just some library imports for Plotly so we can make use of the interactive graphs.
import plotly.express as px
import plotly.io as pio

pio.renderers.default = "vscode"

## Writing Data
The first step is to write data into InfluxDB. We will use the `write_api` to write data into InfluxDB. In this example we are going to utilise `batching mode` to write data in batches. This is the most efficient way to write data into InfluxDB. To do this we are going to first setup some paramters for our client.

## Client Setup
Now that we have done the inital configurations of our write paramters its time to include these within our client initalization. The InfluxDB 3.0 Client can both write and query data. For now we will use it to write data based upon our configuration.

In [ ]:
# In this example we are using the InfluxDBClient3 object to connect to the InfluxDB Cloud instance.
# We are also passing in the write_client_options we created earlier.
# The token, host and org are all required to connect to the InfluxDB Serverless instance.
# Note: that Org is optional with Dedicated instances.


client = FlightClient("grpc://localhost:8081")



now =  pd.Timestamp.now(tz='+00:00').floor('ms')

# Lists of possible trainers
trainers = ["ash", "brock", "misty", "gary", "jessie", "james"]

# Read the CSV into a DataFrame. (Credit to @ritchie46 for the dataset)
pokemon_df = pd.read_csv("./pokemon.csv")

# Creating an empty list to store the data
data = []

# Dictionary to keep track of the number of times each trainer has caught each Pokémon
trainer_pokemon_counts = {}

# Number of entries we want to create
num_entries = 1000

### Generate some data
Now that we have our client setup lets start generating some data we can write to InfluxDB. Following the Pokemon example we will create a list of trainers and the number of pokemon they have caught. Trainers will catch pokemon randomly selected from our list stored within the Pandas DataFrame `pokemon_df`.

In [ ]:
from IPython.display import display, HTML

# Generating random data
for i in range(num_entries):
    trainer = random.choice(trainers)
    
    # Randomly select a row from pokemon_df
    random_pokemon = pokemon_df.sample().iloc[0]
    caught = random_pokemon['Name']
    
    # Count the number of times this trainer has caught this Pokémon
    if (trainer, caught) in trainer_pokemon_counts:
        trainer_pokemon_counts[(trainer, caught)] += 1
    else:
        trainer_pokemon_counts[(trainer, caught)] = 1
    
    # Get the number for this combination of trainer and Pokémon
    num = trainer_pokemon_counts[(trainer, caught)]

    entry = {
        "trainer": trainer,
        "id": f"{0000 + random_pokemon['#']:04d}",
        "num": str(num),
        "name": caught,
        "level": random.randint(5, 20),
        "attack": random_pokemon['Attack'],
        "defense": random_pokemon['Defense'],
        "hp": random_pokemon['HP'],
        "speed": random_pokemon['Speed'],
        "type1": random_pokemon['Type 1'],
        "type2": random_pokemon['Type 2'],
        "timestamp": now
    }
    data.append(entry)

# Convert the list of dictionaries to a DataFrame
caught_pokemon_df = pd.DataFrame(data).set_index('timestamp')

# Print the DataFrame
display(caught_pokemon_df)

### Write Data to FDAP

In [ ]:
import time
import pyarrow as pa

# Convert the DataFrame to an Arrow Table
table = pa.Table.from_pandas(caught_pokemon_df)

try:
    descriptor = FlightDescriptor.for_path("pokemon")
    writer, _ = client.do_put(descriptor, table.schema)
    writer.write_table(table)
    print(f"wrote: {table}")
    writer.close()
except Exception as e:
    print(f"Error writing point: {e}")



## Querying Data
We have now stored 1000 registered pokemon catches within InfluxDB. We can now query this data using the InfluxDB 3.0 Python Client to gain some insights into our data. We are going to use Plotly to visualise our data.

In [ ]:

import json
from pyarrow.flight import Ticket



# Lets start with a simple query to understand our schema.
query = '''SHOW COLUMNS FROM pokemon'''

# We can use the query method to run a query against the database.
# Under the hood this creates a flight ticket and uses the FlightClient to run the query. 
# For this example we are using the pandas mode, which will return a pandas DataFrame.
# Language also allows us to specify the query language, in this case we are using SQL.
ticket_bytes = json.dumps({'sql':query, 'table':'pokemon'})
ticket = Ticket(ticket_bytes)
reader = client.do_get(ticket)
df = reader.read_all().to_pandas()

display(df)

### Simple Aggregation
The first query we will run is a simple SQL query to get the number of pokemon caught by each trainer. We will then use Plotly to visualise this data.

In [ ]:
# Lets start with a simple query to understand our schema.
query = '''SELECT count("name") as count, trainer FROM pokemon GROUP BY trainer'''

ticket_bytes = json.dumps({'sql':query, 'table':'pokemon'})
ticket = Ticket(ticket_bytes)
reader = client.do_get(ticket)
table = reader.read_all()
print(table)
df = table.to_pandas()

fig1 = px.bar(df, x="trainer", y="count",color='trainer' ,title='Number of Pokémon caught in the last hour')
fig1.show()

In [ ]:
# Lets start with a simple query to understand our schema.
query = '''SELECT count("name") as count, trainer, "type1" FROM pokemon GROUP BY "trainer","type1"'''

ticket_bytes = json.dumps({'sql':query, 'table':'pokemon'})
ticket = Ticket(ticket_bytes)
reader = client.do_get(ticket)
df = reader.read_all().to_pandas()


# We can use the query method to run a query against the database.
# Under the hood this creates a flight ticket and uses the FlightClient to run the query. 
# For this example we are using the pandas mode, which will return a pandas DataFrame.
# Language also allows us to specify the query language, in this case we are using SQL.

fig2 = px.bar(df, x="trainer", y="count", color='type1', barmode= 'group', title='Number of Pokémon caught grouped by type')
fig2.show()

### Working with Arrow tables
So within the last section we discussed converting returned queries directly to Pandas Dataframes. However, we can also utilise their raw format, Arrow tables. Arrow tables are a columnar format that is more efficient for working with data.

Lets first start by adding more data with a random timestamp between now and 1 hour ago

In [ ]:
num_entries = 1000000 # You can reduce this if required. The more data the more interesting the results.
now =  pd.Timestamp.now(tz='+00:00').floor('ns')
data = []


# Generating random data
for i in range(num_entries):
    # Randomise the timestamp
    timestamp = now - pd.Timedelta(minutes=random.randint(0, 60))
    trainer = random.choice(trainers)
    
    # Randomly select a row from pokemon_df
    random_pokemon = pokemon_df.sample().iloc[0]
    caught = random_pokemon['Name']
    
    # Count the number of times this trainer has caught this Pokémon
    if (trainer, caught) in trainer_pokemon_counts:
        trainer_pokemon_counts[(trainer, caught)] += 1
    else:
        trainer_pokemon_counts[(trainer, caught)] = 1
    
    # Get the number for this combination of trainer and Pokémon
    num = trainer_pokemon_counts[(trainer, caught)]

    entry = {
        "trainer": trainer,
        "id": f"{0000 + random_pokemon['#']:04d}",
        "num": str(num),
        "name": caught,
        "level": random.randint(5, 20),
        "attack": random_pokemon['Attack'],
        "defense": random_pokemon['Defense'],
        "hp": random_pokemon['HP'],
        "speed": random_pokemon['Speed'],
        "type1": random_pokemon['Type 1'],
        "type2": random_pokemon['Type 2'],
        "timestamp": timestamp
    }
    data.append(entry)

# Convert the list of dictionaries to a DataFrame
caught_pokemon_df = pd.DataFrame(data).set_index('timestamp')

# Print the DataFrame
display(caught_pokemon_df)
table = pa.Table.from_pandas(caught_pokemon_df)

try:
    descriptor = FlightDescriptor.for_path("caught")
    writer, _ = client.do_put(descriptor, table.schema)
    writer.write_table(table)
    print(f"wrote: {table}")
    writer.close()
except Exception as e:
    print(f"Error writing point: {e}")



Now lets return the result as an Arrow table. We can recreate he same aggregation we did on server side with the pyarrow library. the `mode='all'` parameter tells InfluxDB to return all data within the query result as a Arrow table. We can then use the `to_pandas()` method to convert the Arrow table to a Pandas DataFrame.

In [ ]:
query='''SELECT * FROM caught'''
ticket_bytes = json.dumps({'sql':query, 'table':'caught'})
ticket = Ticket(ticket_bytes)
reader = client.do_get(ticket)
table = reader.read_all()


# PyArrow Aggregation
aggregation = table.group_by(["trainer", "type1"]).aggregate([("name", "count")]).to_pandas()
fig3 = px.bar(aggregation, x="trainer", y="name_count", color='type1', barmode= 'group', title='Number of Pokémon caught grouped by type')
fig3.show()

### Saving to file
We can also save our query results to file. This is useful if we want to save our data for later use. We can save our data in a number of formats including CSV, JSON, Parquet and Apache Arrow. Lets save our data as a parquet file.

In [ ]:
import pyarrow.parquet as pq

# Write the table to a parquet file
pq.write_table(table, 'kanto.parquet')

### Working with Parquet files
This allows to show show off another feature of the InfluxDB 3.0 Python Client. We can parse our file directly to InfluxDB. This is useful if we want to upload data from a local file. Lets upload our parquet file to InfluxDB.

In [ ]:
# Read the parquet file into a table
table = pq.read_table('kanto.parquet')
descriptor = FlightDescriptor.for_path("kanto")
writer, _ = client.do_put(descriptor, table.schema)
writer.write_table(table)
print(f"wrote: {table}")
writer.close()


### Group by time query
Finally lets run a group by time query to get the number of pokemon caught by each trainer over the last hour grouped into 10 minute intervals. We will then use Plotly to visualise this data.

In [ ]:
query='''SELECT trainer,
  date_bin(INTERVAL '1 minute', timestamp, TIMESTAMP '1970-01-01 00:00:00Z') AS bin,
  count("name") AS count
FROM caught
GROUP BY bin, trainer
ORDER BY bin DESC'''

ticket_bytes = json.dumps({'sql':query, 'table':'caught'})
ticket = Ticket(ticket_bytes)
reader = client.do_get(ticket)
df = reader.read_all().to_pandas()
print(df)

fig4 = px.line(df, x="bin", y="count", color='trainer', title='Number of Pokémon caught in the last hour grouped by trainer and time')
fig4.show()

## Pandas and Polars

In [ ]:
import time
import pandas as pd
import polars as pl

class CodeTimer:
    def __init__(self):
        self.start_time = None
        self.end_time = None

    def start(self):
        self.start_time = time.time()

    def stop(self):
        if self.start_time is None:
            raise Exception("Timer has not been started. Use .start() method to start it.")

        self.end_time = time.time()
        elapsed_time = self.end_time - self.start_time
        print(f"Elapsed time: {elapsed_time} seconds")



query='''SELECT timestamp, attack FROM caught'''
ticket_bytes = json.dumps({'sql':query, 'table':'caught'})
ticket = Ticket(ticket_bytes)
reader = client.do_get(ticket)
table = reader.read_all()

print("Pandas == Arrow Table Conversion")
timer = CodeTimer()
timer.start()
df = table.to_pandas()
timer.stop()
df = df.set_index("timestamp")


print("Polars == Arrow table Conversion ")
timer = CodeTimer()
timer.start()
pdf = pl.from_arrow(table)
timer.stop()

print("Pandas == Resampling")
timer = CodeTimer()
timer.start()
result_pandas = df.resample('1T').mean()
timer.stop()

pdf = pdf.sort("timestamp")
print("Polars == Resampling")
timer = CodeTimer()
timer.start()
result_polars = pdf.group_by_dynamic(
    index_column='timestamp', 
    every='1h', 
    closed='left', 
    include_boundaries=True
).agg([
    pl.col('attack').mean().alias('mean_value')
])
timer.stop()

print("Pandas == to Arrow ")
timer = CodeTimer()
timer.start()
table = pa.Table.from_pandas(result_pandas)
timer.stop()

print("Polars == to Arrow")
timer = CodeTimer()
timer.start()
table=result_polars.to_arrow()
timer.stop()

# Conclusion
We have now covered the basics of the InfluxDB 3.0 Python Client. I hope you found this novel cook book informative and fun. If you have any questions, bugs or feature requests please raise an issue on the [GitHub repo](https://github.com/InfluxCommunity/influxdb3-python/issues)


<p align="center">
<img height="100", width="100" src="https://i.pinimg.com/originals/18/15/44/181544facabe62d30c52e94b369f0f3a.png">
</p>